In [1]:
import subprocess
import os

import pandas as pd
import numpy as np

from datetime import datetime

import requests
import json

In [2]:
def get_johns_hopkins():
    '''GEt full data from git pull request stored in csv file.'''
    git_pull = subprocess.Popen( "git pull" ,
                         cwd = os.path.dirname('../../data/raw/COVID-19/' ),
                         shell = True,
                         stdout = subprocess.PIPE,
                         stderr = subprocess.PIPE )
    (out, error) = git_pull.communicate()


    print("Error : " + str(error))
    print("out : " + str(out))

In [3]:
def get_current_data_germany():
    ''' Get current data from germany. FInal data frame is stored as pd.DataFrame
    '''
    # 16 states
    #data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

    # 400 regions / Landkreise
    data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_Landkreisdaten/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

    json_object=json.loads(data.content)
    full_list=[]
    for pos,each_dict in enumerate (json_object['features'][:]):
        full_list.append(each_dict['attributes'])

    pd_full_list=pd.DataFrame(full_list)
    pd_full_list.to_csv('../../data/raw/NPGEO/GER_state_data.csv',sep=';')
    print(' Number of regions rows: '+str(pd_full_list.shape[0]))
    print(type(pd_full_list))
    print(pd_full_list.columns)

In [4]:
if __name__ == '__main__':
    get_johns_hopkins()
    get_current_data_germany()

Error : b'From https://github.com/CSSEGISandData/COVID-19\n   24587a0b6f..df7d576485  web-data   -> origin/web-data\n'
out : b'Already up to date.\n'
 Number of regions rows: 411
<class 'pandas.core.frame.DataFrame'>
Index(['OBJECTID', 'ADE', 'GF', 'BSG', 'RS', 'AGS', 'SDV_RS', 'GEN', 'BEZ',
       'IBZ', 'BEM', 'NBD', 'SN_L', 'SN_R', 'SN_K', 'SN_V1', 'SN_V2', 'SN_G',
       'FK_S3', 'NUTS', 'RS_0', 'AGS_0', 'WSK', 'EWZ', 'KFL', 'DEBKG_ID',
       'Shape__Area', 'Shape__Length', 'death_rate', 'cases', 'deaths',
       'cases_per_100k', 'cases_per_population', 'BL', 'BL_ID', 'county',
       'last_update', 'cases7_per_100k', 'recovered', 'EWZ_BL',
       'cases7_bl_per_100k', 'cases7_bl', 'death7_bl', 'cases7_lk',
       'death7_lk', 'cases7_per_100k_txt', 'AdmUnitId'],
      dtype='object')


 Number of regions rows: 411
<class 'pandas.core.frame.DataFrame'>
Index(['OBJECTID', 'ADE', 'GF', 'BSG', 'RS', 'AGS', 'SDV_RS', 'GEN', 'BEZ',
       'IBZ', 'BEM', 'NBD', 'SN_L', 'SN_R', 'SN_K', 'SN_V1', 'SN_V2', 'SN_G',
       'FK_S3', 'NUTS', 'RS_0', 'AGS_0', 'WSK', 'EWZ', 'KFL', 'DEBKG_ID',
       'Shape__Area', 'Shape__Length', 'death_rate', 'cases', 'deaths',
       'cases_per_100k', 'cases_per_population', 'BL', 'BL_ID', 'county',
       'last_update', 'cases7_per_100k', 'recovered', 'EWZ_BL',
       'cases7_bl_per_100k', 'cases7_bl', 'death7_bl', 'cases7_lk',
       'death7_lk', 'cases7_per_100k_txt', 'AdmUnitId'],
      dtype='object')


In [32]:
import pandas as pd
import numpy as np

from datetime import datetime

In [69]:
def store_relational_JH_dataset():
    ''' Transformes the COVID data into a relational data set.'''

    data_path='../../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
    pd_raw=pd.read_csv(data_path)

    pd_data_base=pd_raw.rename(columns={'Country/Region':'country','Province/State':'state'})

    pd_data_base['state']=pd_data_base['state'].fillna('no')

    pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)
    #print(pd_data_base.columns)
    #print('-------------')
    #print(pd_data_base.set_index(['state','country']).head(3))
    #print('-------------')
    #print(pd_data_base.set_index(['state','country']).T.head(3))
    #print('-------------')
    #print(pd_data_base.set_index(['state','country']).T.stack(level=[0,1]).head(30))
    #print('-------------')
    #print(pd_data_base.set_index(['state','country']).T.stack(level=[0,1]).reset_index().head(30))
    #print('-------------')
    
    pd_relational_model=pd_data_base.set_index(['state','country']) \
                                .T                              \
                                .stack(level=[0,1])             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'confirmed'},
                                                  )

    pd_relational_model['date']=pd_relational_model.date.astype('datetime64[ns]')
    print(pd_relational_model.columns)
    print(pd_relational_model.head(10))
    pd_relational_model.to_csv('../../data/processed/COVID_relational_confirmed.csv',sep=';',index=False)
    print(' Number of rows stored: '+str(pd_relational_model.shape[0]))


In [70]:
if __name__ == '__main__':

    store_relational_JH_dataset()

Index(['date', 'state', 'country', 'confirmed'], dtype='object')
        date                             state         country  confirmed
0 2020-01-22                           Alberta          Canada        0.0
1 2020-01-22                          Anguilla  United Kingdom        0.0
2 2020-01-22                             Anhui           China        1.0
3 2020-01-22                             Aruba     Netherlands        0.0
4 2020-01-22      Australian Capital Territory       Australia        0.0
5 2020-01-22                           Beijing           China       14.0
6 2020-01-22                           Bermuda  United Kingdom        0.0
7 2020-01-22  Bonaire, Sint Eustatius and Saba     Netherlands        0.0
8 2020-01-22                  British Columbia          Canada        0.0
9 2020-01-22            British Virgin Islands  United Kingdom        0.0
 Number of rows stored: 261630
